In [ ]:
#import libraries

import pandas as pd

from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords

nltk.download('stopwords')  # Ensure it's downloaded

import re

import contractions

from sklearn.feature_extraction.text import TfidfVectorizer #to vectorize the tokenized words

from sklearn.linear_model import LogisticRegression #model chosen for training
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kathe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kathe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kathe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk #repeat import becuase it gives error sometimes
nltk.download('punkt')

# Check where nltk is looking for data
print(nltk.data.path)



['C:\\Users\\Kathe/nltk_data', 'c:\\Users\\Kathe\\AppData\\Local\\Programs\\Python\\Python313\\nltk_data', 'c:\\Users\\Kathe\\AppData\\Local\\Programs\\Python\\Python313\\share\\nltk_data', 'c:\\Users\\Kathe\\AppData\\Local\\Programs\\Python\\Python313\\lib\\nltk_data', 'C:\\Users\\Kathe\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kathe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
import nltk
nltk.data.path.append("C:/Users/Kathe/nltk_data")  # Change to your actual path


In [ ]:
stops = set(stopwords.words('english')) #create the list of stopwords to remove from title and body text
print(stops)

{'each', 'between', 'am', 'are', 'too', 'this', 'nor', 'his', 'from', "i'll", 'having', 'who', 'you', 'against', 'here', 'being', "doesn't", 'most', 'off', "i'm", 'by', 'which', "you're", 'will', "she'd", "hasn't", 'why', 'up', 'hadn', 'your', "couldn't", 'theirs', 'whom', 'that', 'the', 'own', "i've", 'has', 've', 'how', 'll', 'few', 'hasn', 're', 'does', 'such', "they're", "weren't", "isn't", 't', 'no', 'needn', 'during', 'him', 'd', 'some', 'because', "it's", 'they', 'when', 'her', "won't", 'those', "he's", "he'd", "you'll", 'what', 'under', 'ourselves', 'doing', 'any', 'more', "we'll", 'all', 'hers', 'or', 'a', 'but', "they'd", 'in', 'until', 'only', 'can', "should've", 'to', "haven't", 'himself', 'did', 's', "shouldn't", "don't", 'aren', 'mightn', "they've", 'we', 'into', 'after', 'below', "you'd", 'them', "she's", 'where', 'doesn', 'been', 'me', 'wasn', 'my', 'isn', 'itself', 'had', "mightn't", 'so', 'he', "mustn't", "it'll", "he'll", 'won', 'just', 'ain', "they'll", "that'll", '

punkt:
This dataset is used for tokenization—the process of splitting text into individual words or sentences. The 'punkt' tokenizer is a pre-trained model that recognizes common patterns in text, such as punctuation marks and sentence boundaries. It helps the tokenizer understand where words or sentences begin and end.

Pwordnet:
This dataset provides access to WordNet, a large lexical database of English. WordNet groups English words into sets of synonyms (synsets), which helps in tasks like lemmatization. The WordNet lemmatizer uses this dataset to reduce words to their base or root form, like converting "running" to "run" and "better" to "good."


In [ ]:
def leer_csv(name): #function to read csv
    return pd.read_csv(name)

In [4]:
datasets = ['dataset_1.csv', 'dataset_3.csv', 'dataset_4.csv', 'dataset_5.csv', 'dataset_6.csv'] #list of datasets

In [5]:
for set in datasets:
    leer_csv(set)
    print(leer_csv(set).head()) #print the first 5 rows of each dataset

                                               title  \
0                       You Can Smell Hillary’s Fear   
1  Watch The Exact Moment Paul Ryan Committed Pol...   
2        Kerry to go to Paris in gesture of sympathy   
3  Bernie supporters on Twitter erupt in anger ag...   
4   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL  
                                               title  \
0         Four ways Bob Corker skewered Donald Trump   
1  Linklater's war veteran comedy speaks to moder...   
2  Trump’s Fight With Corker Jeopardizes His Legi...   
3  Egypt's Cheiron wins tie-up with Pemex for Mex...   
4        Jason Al

In [6]:
for set in datasets:
    print(leer_csv(set).shape) #print the shape of each dataset

(6306, 3)
(3522, 3)
(39105, 3)
(3522, 3)
(799, 3)


In [7]:
for set in datasets:
    print(leer_csv(set).duplicated().sum()) #print the number of duplicated rows in each dataset

0
0
0
0
0


In [8]:
for set in datasets:
    print(leer_csv(set).columns) #print the columns of each dataset, enusre that the columns are the same

Index(['title', 'text', 'label'], dtype='object')
Index(['title', 'text', 'label'], dtype='object')
Index(['title', 'text', 'label'], dtype='object')
Index(['title', 'text', 'label'], dtype='object')
Index(['title', 'text', 'label'], dtype='object')


In [9]:
for set in datasets:
    df = leer_csv(set)
    print(f"Unique values in {set}:")
    print(df['label'].unique()) #print the unique values in the label column of each dataset, ensure uniformity


Unique values in dataset_1.csv:
['FAKE' 'REAL']
Unique values in dataset_3.csv:
['REAL' 'FAKE']
Unique values in dataset_4.csv:
['FAKE' 'REAL']
Unique values in dataset_5.csv:
['REAL' 'FAKE']
Unique values in dataset_6.csv:
['FAKE' 'REAL']


In [10]:
for set in datasets:
    df = leer_csv(set)
    print(f"Null values in {set}:")
    print(df.isnull().sum()) #print the number of null values in each dataset, one value detected in dataset_3

Null values in dataset_1.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_3.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_4.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_5.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_6.csv:
title    0
text     0
label    0
dtype: int64


In [11]:
df_dataset_3 = leer_csv('dataset_3.csv') #read dataset_3, drop the null value and save the cleaned dataset
df_dataset_3= df_dataset_3.dropna()
df_dataset_3.to_csv('dataset_3.csv', index=False)

In [12]:
for set in datasets: #confirm no remaining null values
    df = leer_csv(set)
    print(f"Null values in {set}:")
    print(df.isnull().sum())

Null values in dataset_1.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_3.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_4.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_5.csv:
title    0
text     0
label    0
dtype: int64
Null values in dataset_6.csv:
title    0
text     0
label    0
dtype: int64


In [13]:
df_list = []  # Initialize an empty list to store DataFrames

for dataset in datasets:  # Loop through each file name
    df_list.append(leer_csv(dataset))  # Append the DataFrame to the list

df_combined = pd.concat(df_list, ignore_index=True)  # Merge all DataFrames

print(df_combined.shape)  # Check the size of the final dataset


(53254, 3)


In [14]:
df_combined['text'].unique()

array(['Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that th

In [ ]:
def clean_text(columna):
    """Function to clean text in the given column of df_combined."""
    
    # Convert to string first (to avoid NaN issues)
    df_combined[columna] = df_combined[columna].astype(str)

    # Remove dates like 01-01-2020
    pattern_date = r"\b\d{2}-\d{2}-\d{4}\b"
    df_combined[columna] = df_combined[columna].str.replace(pattern_date, "", regex=True).str.strip()

    # Remove dates like January 01 2020 (or Jan 01 2020)
    pattern_date2 = r"\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\s\d{1,2}\s\d{4}\b"
    df_combined[columna] = df_combined[columna].str.replace(pattern_date2, "", regex=True).str.strip()

    # Remove location prefixes like "Newark, N.J. (Reuters) -", "Paris (Reuters) -", etc.
    pattern_location = r"[a-z\s]+(?:,\s?[a-z.\s]+)?\s\([a-z\s]+\)\s?-\s?"
    df_combined[columna] = df_combined[columna].str.replace(pattern_location, "", regex=True).str.strip()

    # Expand contractions (e.g., "can't" -> "cannot")
    df_combined[columna] = df_combined[columna].apply(lambda x: contractions.fix(x))

    # Remove stopwords
    df_combined[columna] = df_combined[columna].apply(lambda x: " ".join([word for word in x.split() if word.lower() not in stops]))

    # Convert text to lowercase at the end
    df_combined[columna] = df_combined[columna].str.lower()

    # Restore "U.S."
    df_combined[columna] = df_combined[columna].str.replace("you.s", "u.s", regex=True)

    return df_combined



In [26]:
columns_to_clean = ['text', 'title']  # Define the columns to consider

for column in columns_to_clean:  # Loop through each column
    clean_text(column)  # Clean the text

In [18]:
pattern = r"^[a-z\s]+(?:,\s?[a-z.\s]+)?\s\([a-z\s]+\)\s?-\s?"
# Apply regex to clean the text column
df_combined['text'] = df_combined['text'].apply(lambda x: re.sub(pattern, "", x).strip())

In [19]:
df_combined.sample(15)

,title,text,label
29380,republican leader plans senate vote healthcare...,senate republican leader mitch mcconnell plans...,REAL
45170,fdp's push invite putin g7 sows discord within...,leader germany pro-business free democrats (fd...,REAL
29321,china says ban petroleum exports north korea,china said saturday ban exports petroleum prod...,REAL
20143,lgbt volunteers waiting thrown rooftop…join fi...,group volunteer soldiers announced week first ...,FAKE
12409,look japanese leader’s face meeting trump says...,alleged president donald trump met japanese pr...,FAKE
22817,scoundrel hillary supporter starts “trumpleaks...,hillary clinton ally david brock offering pay ...,FAKE
26437,kellyanne conway delivers knock punch smug jak...,audience member asks kellyanne conway trump co...,FAKE
20779,breaking: french vote mass muslim immigration ...,french citizens voted demise elected macron na...,FAKE
17812,cruz leaves closet disappointed secret meeting...,"gets creepier creepier ted cruz. time, took be...",FAKE
713,"left turns bob dylan pro-israel views, refusal...","left turns bob dylan pro-israel views, refusal...",FAKE


In [20]:
df_combined['title'].unique()

array(['smell hillary’s fear',
       'watch exact moment paul ryan committed political suicide trump rally (video)',
       'kerry go paris gesture sympathy', ...,
       'chemical attack kills five syrians aleppo sana',
       '5 killed russian military chopper shot syria',
       'syrian army kills 48 isil terrorists deir ezzor'],
      shape=(48339,), dtype=object)

In [ ]:
# create train and test datasets
train_df, test_df = train_test_split(df_combined, test_size=0.2, random_state=42) #test size .02 indicates a 80/20 split


In [22]:
print(f'train shape: {train_df.shape}, test shape: {test_df.shape}')  # Check the size of the final datasets

train shape: (42603, 3), test shape: (10651, 3)


In [24]:
# Save for later use
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [23]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [24]:
train_df.isnull().sum()


title      0
text     383
label      0
dtype: int64

In [25]:
#drop rows with null values
train_df = train_df.dropna(subset=['text'])


In [26]:
train_df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [14]:
test_df.isnull().sum()

title               0
text              100
label               0
tokenized_text      0
dtype: int64

In [27]:
test_df = test_df.dropna(subset=['text'])

In [28]:
test_df.isnull().sum()

title    0
text     0
label    0
dtype: int64

Tokenization: word_tokenize function from the nltk.tokenize module to break the input text into individual words (tokens). For example, "I love coding!" would become ['I', 'love', 'coding', '!'].

Lemmatization: reduces a word to its base or root form. For instance, "running" would become "run" and "better" would become "good". Uses WordNetLemmatizer from the nltk.stem module. 



In [44]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if text is a string
        tokens = word_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        # Lemmatize while keeping only alphabetic words using list comprehension
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
        return tokens
    else:
        return []  # Return an empty list if the input is not a string



In [11]:
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kathe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kathe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [30]:
train_df['tokenized_text'] = train_df['text'].apply(preprocess_text)
test_df['tokenized_text'] = test_df['text'].apply(preprocess_text)


In [36]:
train_df.head()

,title,text,label,tokenized_text
0,puerto rican police officer: san juan mayor al...,puerto rican police officer: san juan mayor al...,FAKE,"[puerto, rican, police, officer, san, juan, ma..."
1,brazil's president recovering prostate surgery,brazilian president michel temer recovering su...,REAL,"[brazilian, president, michel, temer, recoveri..."
2,white house facing rocky legal road immigration,"campaign, trump threatened impose large tariff...",REAL,"[campaign, trump, threatened, impose, large, t..."
3,"watch: newt gingrich admits trump care unity, ...",democrats offering work donald trump cease des...,FAKE,"[democrat, offering, work, donald, trump, ceas..."
4,trump outlines treat america like bankrupt bus...,furniture company southern maine called chapte...,FAKE,"[furniture, company, southern, maine, called, ..."


In [37]:
test_df.head()

,title,text,label,tokenized_text
0,bet hip hop awards 2017: winners list,story highlights cardi b. performed also singl...,REAL,"[story, highlight, cardi, performed, also, sin..."
1,hippie throwback: video photos emerge socialis...,pathetic little basement dweller original occu...,FAKE,"[pathetic, little, basement, dweller, original..."
2,"louisiana’s dem. governor steps up, signs exec...","could anyone, million years, ever see bobby ji...",FAKE,"[could, anyone, million, year, ever, see, bobb..."
3,"police: kalamazoo spree killer uber driver, pi...","disturbing twist already-terrifying story, man...",FAKE,"[disturbing, twist, story, man, believed, behi..."
4,[video] leftist cnn host actually said this: c...,shamelessly bashing people think human action ...,FAKE,"[shamelessly, bashing, people, think, human, a..."


In [31]:
print(type(train_df))  # Should output: <class 'pandas.core.frame.DataFrame'>
print(type(test_df))   # Should output: <class 'pandas.core.frame.DataFrame'>


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [32]:
print(train_df.columns)  # Check if 'text' is in the columns of train_df
print(test_df.columns)   # Check if 'text' is in the columns of test_df


Index(['title', 'text', 'label', 'tokenized_text'], dtype='object')
Index(['title', 'text', 'label', 'tokenized_text'], dtype='object')


In [33]:
print(train_df['text'].isnull().sum())  # Check if there are any NaN values in 'text' column of train_df
print(test_df['text'].isnull().sum())   # Check if there are any NaN values in 'text' column of test_df


0
0


In [45]:
train_df['tokenized_title'] = train_df['title'].apply(preprocess_text)
test_df['tokenized_title'] = test_df['title'].apply(preprocess_text)


In [46]:
train_df.head()

,title,text,label,tokenized_text,tokenized_title
0,puerto rican police officer: san juan mayor al...,puerto rican police officer: san juan mayor al...,FAKE,"[puerto, rican, police, officer, san, juan, ma...","[puerto, rican, police, officer, san, juan, ma..."
1,brazil's president recovering prostate surgery,brazilian president michel temer recovering su...,REAL,"[brazilian, president, michel, temer, recoveri...","[brazil, president, recovering, prostate, surg..."
2,white house facing rocky legal road immigration,"campaign, trump threatened impose large tariff...",REAL,"[campaign, trump, threatened, impose, large, t...","[white, house, facing, rocky, legal, road, imm..."
3,"watch: newt gingrich admits trump care unity, ...",democrats offering work donald trump cease des...,FAKE,"[democrat, offering, work, donald, trump, ceas...","[watch, newt, gingrich, admits, trump, care, u..."
4,trump outlines treat america like bankrupt bus...,furniture company southern maine called chapte...,FAKE,"[furniture, company, southern, maine, called, ...","[trump, outline, treat, america, like, bankrup..."


In [47]:
test_df.head()

,title,text,label,tokenized_text,tokenized_title
0,bet hip hop awards 2017: winners list,story highlights cardi b. performed also singl...,REAL,"[story, highlight, cardi, performed, also, sin...","[bet, hip, hop, award, winner, list]"
1,hippie throwback: video photos emerge socialis...,pathetic little basement dweller original occu...,FAKE,"[pathetic, little, basement, dweller, original...","[hippie, throwback, video, photo, emerge, soci..."
2,"louisiana’s dem. governor steps up, signs exec...","could anyone, million years, ever see bobby ji...",FAKE,"[could, anyone, million, year, ever, see, bobb...","[louisiana, s, dem, governor, step, up, sign, ..."
3,"police: kalamazoo spree killer uber driver, pi...","disturbing twist already-terrifying story, man...",FAKE,"[disturbing, twist, story, man, believed, behi...","[police, kalamazoo, spree, killer, uber, drive..."
4,[video] leftist cnn host actually said this: c...,shamelessly bashing people think human action ...,FAKE,"[shamelessly, bashing, people, think, human, a...","[video, leftist, cnn, host, actually, said, th..."


In [48]:
train_df.to_csv("train_data_tokenized.csv", index=False)
test_df.to_csv("test_data_tokenized.csv", index=False)

TF-IDF (Term Frequency-Inverse Document Frequency): transform text data into numerical vectors that capture the importance of words in relation to the corpus.

With max_features=n: The vectorizer will only keep the top n most important words based on their TF-IDF score (i.e., the highest words according to the frequency of term occurrence relative to its rarity in the corpus). This helps reduce the dimensionality of the data, making the model more efficient and often improving generalization.

In [50]:
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed

# Apply TF-IDF vectorization to the tokenized text (you might want to rejoin the tokens into strings before this step)
train_df['text_str'] = train_df['tokenized_text'].apply(lambda x: ' '.join(x))
test_df['text_str'] = test_df['tokenized_text'].apply(lambda x: ' '.join(x))

X_train_text = vectorizer.fit_transform(train_df['text_str'])
X_test_text = vectorizer.transform(test_df['text_str'])

# Apply TF-IDF vectorization to the tokenized titles (similar approach as for text)
train_df['title_str'] = train_df['tokenized_title'].apply(lambda x: ' '.join(x))
test_df['title_str'] = test_df['tokenized_title'].apply(lambda x: ' '.join(x))

X_train_title = vectorizer.fit_transform(train_df['title_str'])
X_test_title = vectorizer.transform(test_df['title_str'])

Selection of Logistic Regression as training model,principally for the simplicity. 

In [ ]:
#horizontal stack the text and title feature matrices
X_train_combined = hstack([X_train_text, X_train_title])
X_test_combined = hstack([X_test_text, X_test_title])

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train_combined, train_df['label'])

# Predict on the test set
y_pred = model.predict(X_test_combined)

# Evaluate the model
print("Training Accuracy:", accuracy_score(train_df['label'], model.predict(X_train_combined)))
print("Test Accuracy:", accuracy_score(test_df['label'], y_pred))

Training Accuracy: 0.9704642349597348
Test Accuracy: 0.9541275708463652
